# Setup

In [1]:
import os
import sys
import shutil

def remove_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)

In [2]:
!git clone https://github.com/aminshabani/house_diffusion.git /content/HOUSE_DIFFUSION_DIFFUSION

Cloning into '/content/HOUSE_DIFFUSION_DIFFUSION'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 1), reused 21 (delta 1), pack-reused 1
Receiving objects: 100% (32/32), 120.29 KiB | 695.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
%cd /content/HOUSE_DIFFUSION_DIFFUSION
!pip install -r requirements.txt
!pip install e .
!python -m pip install mpi4py
!pip install drawsvg
!pip install cairosvg
!pip install pytorch_fid
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz

/content/HOUSE_DIFFUSION_DIFFUSION
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%mkdir -p /content/HOUSE_DIFFUSION_DIFFUSION/ckpts/exp

In [5]:
!gdown 16zKmtxwY5lF6JE-CJGkRf3-OFoD1TrdR -O /content/HOUSE_DIFFUSION_DIFFUSION/ckpts/exp/model250000.pt

Downloading...
From (original): https://drive.google.com/uc?id=16zKmtxwY5lF6JE-CJGkRf3-OFoD1TrdR
From (redirected): https://drive.google.com/uc?id=16zKmtxwY5lF6JE-CJGkRf3-OFoD1TrdR&confirm=t&uuid=f1c27ec3-d428-4c7e-8ee7-a2322ade343a
To: /content/HOUSE_DIFFUSION_DIFFUSION/ckpts/exp/model250000.pt
100% 106M/106M [00:01<00:00, 75.4MB/s] 


# Data - Preprocess

In [6]:
# %cd /content
# !git clone https://github.com/sepidsh/Housegan-data-reader.git

In [7]:
# !gdown 1roWsHejGZIzawa_KtB4dpquFnjl9w71R -O /content/dataset.rar

In [8]:
# !unrar x /content/dataset.rar
# !mv /content/dataset /content/Housegan-data-reader/rplan_dataset

In [9]:
# def pre_process_data(limit = None):
#     %cd /content/Housegan-data-reader
#     if limit == None:
#       !python3 /content/Housegan-data-reader/run.py
#     else:
#       !python3 /content/Housegan-data-reader/run.py --limit '{limit}'

#     if os.path.exists("/content/HOUSE_DIFFUSION_DIFFUSION/datasets"):
#         remove_folder("/content/HOUSE_DIFFUSION_DIFFUSION/datasets")
#     remove_folder("/content/Housegan-data-reader/failed_rplan_json")

#     %mkdir /content/HOUSE_DIFFUSION_DIFFUSION/datasets
#     %mv /content/Housegan-data-reader/rplan_json /content/HOUSE_DIFFUSION_DIFFUSION/datasets/rplan

In [10]:
# pre_process_data(5000)

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
remove_folder("/content/HOUSE_DIFFUSION_DIFFUSION/datasets")
%mkdir /content/HOUSE_DIFFUSION_DIFFUSION/datasets
%cp -r /content/drive/MyDrive/Final_EA/datasets/rplan /content/HOUSE_DIFFUSION_DIFFUSION/datasets/rplan

# house_diffusion/rplanhg_datasets.py

In [37]:
%cd /content/HOUSE_DIFFUSION_DIFFUSION

/content/HOUSE_DIFFUSION_DIFFUSION


In [38]:
import math
import random
import torch as th

from PIL import Image, ImageDraw
import blobfile as bf
from mpi4py import MPI
import numpy as np
from torch.utils.data import DataLoader, Dataset
from glob import glob
import json
import os
import cv2 as cv
from tqdm import tqdm
from shapely import geometry as gm
from shapely.ops import unary_union
from collections import defaultdict
import copy

In [100]:
def load_rplanhg_data(
    batch_size,
    analog_bit,
    target_set = 8,
    set_name = 'train',
    using_memory = True
):
    """
    For a dataset, create a generator over (shapes, kwargs) pairs.
    """
    print(f"loading {set_name} of target set {target_set}")
    deterministic = False if set_name=='train' else True
    dataset = RPlanhgDataset(set_name, analog_bit, target_set, non_manhattan = True, using_memory = using_memory)

    if deterministic:
        loader = DataLoader(
            dataset, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False
        )
    else:
        loader = DataLoader(
            dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False
        )
    while True:
        yield from loader

In [101]:
def make_non_manhattan(poly, polygon, house_poly):
    dist = abs(poly[2]-poly[0])
    direction = np.argmin(dist)
    center = poly.mean(0)
    min = poly.min(0)
    max = poly.max(0)

    tmp = np.random.randint(3, 7)
    new_min_y = center[1]-(max[1]-min[1])/tmp
    new_max_y = center[1]+(max[1]-min[1])/tmp
    if center[0]<128:
        new_min_x = min[0]-(max[0]-min[0])/np.random.randint(2,5)
        new_max_x = center[0]
        poly1=[[min[0], min[1]], [new_min_x, new_min_y], [new_min_x, new_max_y], [min[0], max[1]], [max[0], max[1]], [max[0], min[1]]]
    else:
        new_min_x = center[0]
        new_max_x = max[0]+(max[0]-min[0])/np.random.randint(2,5)
        poly1=[[min[0], min[1]], [min[0], max[1]], [max[0], max[1]], [new_max_x, new_max_y], [new_max_x, new_min_y], [max[0], min[1]]]

    new_min_x = center[0]-(max[0]-min[0])/tmp
    new_max_x = center[0]+(max[0]-min[0])/tmp
    if center[1]<128:
        new_min_y = min[1]-(max[1]-min[1])/np.random.randint(2,5)
        new_max_y = center[1]
        poly2=[[min[0], min[1]], [min[0], max[1]], [max[0], max[1]], [max[0], min[1]], [new_max_x, new_min_y], [new_min_x, new_min_y]]
    else:
        new_min_y = center[1]
        new_max_y = max[1]+(max[1]-min[1])/np.random.randint(2,5)
        poly2=[[min[0], min[1]], [min[0], max[1]], [new_min_x, new_max_y], [new_max_x, new_max_y], [max[0], max[1]], [max[0], min[1]]]
    p1 = gm.Polygon(poly1)
    iou1 = house_poly.intersection(p1).area/ p1.area
    p2 = gm.Polygon(poly2)
    iou2 = house_poly.intersection(p2).area/ p2.area
    if iou1>0.9 and iou2>0.9:
        return poly
    if iou1<iou2:
        return poly1
    else:
        return poly2

In [102]:
get_bin = lambda x, z: [int(y) for y in format(x, 'b').zfill(z)]
get_one_hot = lambda x, z: np.eye(z)[x]

In [188]:
class RPlanhgDataset(Dataset):
    def __init__(self, set_name, analog_bit, target_set, non_manhattan=False, using_memory = True):
        super().__init__()
        # base_dir = '../datasets/rplan'
        base_dir = "/content/HOUSE_DIFFUSION_DIFFUSION/datasets/rplan"
        self.non_manhattan = non_manhattan
        self.set_name = set_name
        self.analog_bit = analog_bit
        self.target_set = target_set
        self.subgraphs = []
        self.org_graphs = []
        self.org_houses = []
        max_num_points = 100  # số lượng góc tối đa trong một căn nhà

        remove_folder("/content/HOUSE_DIFFUSION_DIFFUSION/processed_rplan")
        %mkdir /content/HOUSE_DIFFUSION_DIFFUSION/processed_rplan
        %cp /content/drive/MyDrive/Final_EA/rplan_train_8_cndist.npz /content/HOUSE_DIFFUSION_DIFFUSION/processed_rplan/rplan_train_8_cndist.npz

        if self.set_name == 'eval':
            if using_memory:
                cnumber_dist = np.load(f'processed_rplan/rplan_train_8_cndist.npz', allow_pickle=True)['cnumber_dist'].item()
            else:
                cnumber_dist = {
                    1: 10,  # living_room
                    2: 4,  # kitchen
                    3: 7,  # bedroom
                    4: 4,  # bathroom
                    5: 4,   # balcony
                    6: 4,   # entrance
                    7: 4,   # dining room
                    8: 4,   # study room
                    10: 4,  # storage
                    11: 4,  # front door
                    12: 4,  # interior_door
                    13: 4   # unknown
                }

        if os.path.exists(f'processed_rplan/rplan_{set_name}_{target_set}.npz'):
            data = np.load(f'processed_rplan/rplan_{set_name}_{target_set}.npz', allow_pickle=True)
            self.graphs = data['graphs']
            self.houses = data['houses']
            self.door_masks = data['door_masks']
            self.self_masks = data['self_masks']
            self.gen_masks = data['gen_masks']
            self.num_coords = 2
            self.max_num_points = max_num_points
            cnumber_dist = np.load(f'processed_rplan/rplan_train_8_cndist.npz', allow_pickle=True)['cnumber_dist'].item()
            if self.set_name == 'eval':
                data = np.load(f'processed_rplan/rplan_{set_name}_{target_set}_syn.npz', allow_pickle=True)
                self.syn_graphs = data['graphs']
                self.syn_houses = data['houses']
                self.syn_door_masks = data['door_masks']
                self.syn_self_masks = data['self_masks']
                self.syn_gen_masks = data['gen_masks']
        else:
            # with open(f'{base_dir}/list.txt') as f:
            #     lines = f.readlines()

            if not os.path.exists('/content/HOUSE_DIFFUSION_DIFFUSION/processed_rplan'):
                %mkdir /content/HOUSE_DIFFUSION_DIFFUSION/processed_rplan

            floorplan_jsons = os.listdir(base_dir)
            cnt=0
            # for line in tqdm(lines):

            # stts = []
            for floorplan_json in floorplan_jsons:
                cnt=cnt+1
                # file_name = f'{base_dir}/{line[:-1]}'
                file_name = os.path.join(base_dir, floorplan_json)

                # print(f'stt: {stt}, file_name: {file_name}')

                rms_type, fp_eds,rms_bbs,eds_to_rms=reader(file_name)
                # print(f'   rms_type: {rms_type}')
                # print(f'   fp_eds: {fp_eds}')
                # print(f'   rms_bbs: {rms_bbs}')
                # print(f'   eds_to_rms: {eds_to_rms}')


                fp_size = len([x for x in rms_type if x != 15 and x != 17])
                if self.set_name=='train' and fp_size == target_set:
                        continue
                if self.set_name=='eval' and fp_size != target_set:
                        continue
                a = [rms_type, rms_bbs, fp_eds, eds_to_rms]
                self.subgraphs.append(a)
                # stts.append(stt)

                # break

            for graph in tqdm(self.subgraphs):
                # print(f'stt: {stt}')

                rms_type = graph[0]
                rms_bbs = graph[1]
                fp_eds = graph[2]
                eds_to_rms= graph[3]

                rms_bbs = np.array(rms_bbs)
                fp_eds = np.array(fp_eds)

                # extract boundary box and centralize
                tl = np.min(rms_bbs[:, :2], 0)
                br = np.max(rms_bbs[:, 2:], 0)
                shift = (tl+br)/2.0 - 0.5

                rms_bbs[:, :2] -= shift
                rms_bbs[:, 2:] -= shift
                fp_eds[:, :2] -= shift
                fp_eds[:, 2:] -= shift

                tl -= shift
                br -= shift

                # build input graph
                graph_nodes, graph_edges, rooms_mks = self.build_graph(rms_type, fp_eds, eds_to_rms)
                # print(f"graph_nodes: {graph_nodes.shape} vs {len(rms_type)}")
                # print(f"graph_edges: {graph_edges.shape} vs {fp_eds.shape}")
                # print(f"graph_edges_instance: {graph_edges[0]}")
                # print(f"rooms_mks: {rooms_mks.shape}")

                house = []
                for room_mask, room_type in zip(rooms_mks, graph_nodes):
                    room_mask = room_mask.astype(np.uint8)
                    room_mask = cv.resize(room_mask, (256, 256), interpolation = cv.INTER_AREA)
                    contours, _ = cv.findContours(room_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
                    contours = contours[0]
                    house.append([contours[:,0,:], room_type])
                self.org_graphs.append(graph_edges)
                # org_graphs lưu toàn bộ graph_edges ứng với từng house
                self.org_houses.append(house)
                # house lưu toàn bộ căn phòng, mỗi căn phòng đc định nghĩa bằng contour (room_coner) và type
                # org_houses lưu toàn bộ house


            houses = []
            door_masks = []
            self_masks = []
            gen_masks = []
            graphs = []
            if self.set_name=='train':
                cnumber_dist = defaultdict(list)

            if self.non_manhattan:
                for h, graph in tqdm(zip(self.org_houses, self.org_graphs), desc='processing dataset'):
                    # Generating non-manhattan Balconies
                    tmp = []
                    for i, room in enumerate(h):
                        if room[1]>10:
                            continue
                        if len(room[0])!=4:
                            continue
                        if np.random.randint(2):
                            continue
                        poly = gm.Polygon(room[0])
                        house_polygon = unary_union([gm.Polygon(room[0]) for room in h])
                        room[0] = make_non_manhattan(room[0], poly, house_polygon)

            for h, graph in tqdm(zip(self.org_houses, self.org_graphs), desc='processing dataset'):
                house = []
                corner_bounds = []
                num_points = 0
                for i, room in enumerate(h):
                    if room[1]>10:
                        room[1] = {15:11, 17:12, 16:13}[room[1]]
                    room[0] = np.reshape(room[0], [len(room[0]), 2])/256. - 0.5 # [[x0,y0],[x1,y1],...,[x15,y15]] and map to 0-1 - > -0.5, 0.5
                    room[0] = room[0] * 2 # map to [-1, 1]
                    ### room[0]_shape = [num_room_corners, 2]

                    # print(f'room[0]: {room[0].shape}')
                    # print(f'room[1]: {room[1]}')

                    if self.set_name=='train':
                        cnumber_dist[room[1]].append(len(room[0]))

                    # Adding conditions
                    num_room_corners = len(room[0])
                    rtype = np.repeat(np.array([get_one_hot(room[1], 25)]), num_room_corners, 0)
                    ### rtype_shape = [num_room_corners, 25]

                    room_index = np.repeat(np.array([get_one_hot(len(house)+1, 32)]), num_room_corners, 0)
                    ### room_index_shape = [num_room_corners, 32]

                    corner_index = np.array([get_one_hot(x, 32) for x in range(num_room_corners)])
                    ### corner_index = [num_room_corners, 32]

                    # Src_key_padding_mask
                    padding_mask = np.repeat(1, num_room_corners)
                    padding_mask = np.expand_dims(padding_mask, 1)
                    ### padding_mask_shape = [num_room_corners, 1]


                    # Generating corner bounds for attention masks
                    connections = np.array([[i,(i+1)%num_room_corners] for i in range(num_room_corners)])
                    connections += num_points
                    ### connections_shape = [num_room_corners, 2]

                    corner_bounds.append([num_points, num_points+num_room_corners])
                    ### corner_bounds thể hiện giới hạn góc của một phòng (về mặt index)
                    num_points += num_room_corners
                    room = np.concatenate((room[0], rtype, corner_index, room_index, padding_mask, connections), 1)
                    house.append(room)

                house_layouts = np.concatenate(house, 0)
                # house_layouts chứa toàn bộ số lượng góc trong căn nhà

                if len(house_layouts)>max_num_points:
                    continue
                padding = np.zeros((max_num_points-len(house_layouts), 94))  # 94 = 2+25+32+32+1+2
                gen_mask = np.ones((max_num_points, max_num_points))
                gen_mask[:len(house_layouts), :len(house_layouts)] = 0
                house_layouts = np.concatenate((house_layouts, padding), 0)

                door_mask = np.ones((max_num_points, max_num_points))
                self_mask = np.ones((max_num_points, max_num_points))
                for i in range(len(corner_bounds)):
                    for j in range(len(corner_bounds)):
                        if i==j:
                            self_mask[corner_bounds[i][0]:corner_bounds[i][1],corner_bounds[j][0]:corner_bounds[j][1]] = 0
                        elif any(np.equal([i, 1, j], graph).all(1)) or any(np.equal([j, 1, i], graph).all(1)):
                            door_mask[corner_bounds[i][0]:corner_bounds[i][1],corner_bounds[j][0]:corner_bounds[j][1]] = 0

                houses.append(house_layouts)
                door_masks.append(door_mask)
                self_masks.append(self_mask)
                gen_masks.append(gen_mask)
                graphs.append(graph)

            self.max_num_points = max_num_points
            self.houses = houses
            self.door_masks = door_masks
            self.self_masks = self_masks
            self.gen_masks = gen_masks
            self.num_coords = 2
            self.graphs = graphs

            # print("max_num_points: ")
            # print(self.max_num_points)

            # print("houses: ")
            # for house in self.houses:
            #     print(house.shape)

            # print("door_masks: ")
            # for door_mask in self.door_masks:
            #     print(door_mask.shape)

            # print("self_masks: ")
            # for self_mask in self.self_masks:
            #     print(self_mask.shape)

            # print("gen_masks: ")
            # for gen_mask in self.gen_masks:
            #     print(gen_mask.shape)

            # print("graphs: ")
            # for graph in self.graphs:
            #     print(graph.shape)



            # np.savez_compressed(f'processed_rplan/rplan_{set_name}_{target_set}', graphs=self.graphs, houses=self.houses,
            #         door_masks=self.door_masks, self_masks=self.self_masks, gen_masks=self.gen_masks)

            np.savez_compressed(f'processed_rplan/rplan_{set_name}_{target_set}',
                    graphs=np.array(self.graphs, dtype=object),
                    houses=np.array(self.houses, dtype=object),
                    door_masks=np.array(self.door_masks, dtype=object),
                    self_masks=np.array(self.self_masks, dtype=object),
                    gen_masks=np.array(self.gen_masks, dtype=object))

            if self.set_name=='train':
                np.savez_compressed(f'processed_rplan/rplan_{set_name}_{target_set}_cndist', cnumber_dist=cnumber_dist)

            if set_name=='eval':
                houses = []
                graphs = []
                door_masks = []
                self_masks = []
                gen_masks = []
                len_house_layouts = 0
                for h, graph in tqdm(zip(self.org_houses, self.org_graphs), desc='processing dataset'):
                    house = []
                    corner_bounds = []
                    num_points = 0

                    if using_memory:
                        num_room_corners_total = [cnumber_dist[room[1]][random.randint(0, len(cnumber_dist[room[1]])-1)] for room in h]
                        while np.sum(num_room_corners_total)>=max_num_points:
                            num_room_corners_total = [cnumber_dist[room[1]][random.randint(0, len(cnumber_dist[room[1]])-1)] for room in h]
                    else:
                        num_room_corners_total = [cnumber_dist[room[1]] for room in h]
                        assert np.sum(num_room_corners_total) < max_num_points

                    for i, room in enumerate(h):
                        # Adding conditions
                        num_room_corners = num_room_corners_total[i]
                        rtype = np.repeat(np.array([get_one_hot(room[1], 25)]), num_room_corners, 0)
                        room_index = np.repeat(np.array([get_one_hot(len(house)+1, 32)]), num_room_corners, 0)
                        corner_index = np.array([get_one_hot(x, 32) for x in range(num_room_corners)])
                        # Src_key_padding_mask
                        padding_mask = np.repeat(1, num_room_corners)
                        padding_mask = np.expand_dims(padding_mask, 1)
                        # Generating corner bounds for attention masks
                        connections = np.array([[i,(i+1)%num_room_corners] for i in range(num_room_corners)])
                        connections += num_points
                        corner_bounds.append([num_points, num_points+num_room_corners])
                        num_points += num_room_corners

                        # print(f"index_0: {np.zeros([num_room_corners, 2]).shape}")
                        # print(f"index_1: {rtype.shape}")
                        # print(f"index_2: {corner_index.shape}")
                        # print(f"index_3: {room_index.shape}")
                        # print(f"index_4: {padding_mask.shape}")
                        # print(f"index_5: {connections.shape}")

                        room = np.concatenate((np.zeros([num_room_corners, 2]), rtype, corner_index, room_index, padding_mask, connections), 1)
                        house.append(room)

                    house_layouts = np.concatenate(house, 0)
                    if np.sum([len(room[0]) for room in h])>max_num_points:
                        continue

                    # print(f"Negative?????: {max_num_points-len(house_layouts)}")

                    padding = np.zeros((max_num_points-len(house_layouts), 94))
                    gen_mask = np.ones((max_num_points, max_num_points))
                    gen_mask[:len(house_layouts), :len(house_layouts)] = 0
                    house_layouts = np.concatenate((house_layouts, padding), 0)

                    door_mask = np.ones((max_num_points, max_num_points))
                    self_mask = np.ones((max_num_points, max_num_points))
                    for i, room in enumerate(h):
                        if room[1]==1:
                            living_room_index = i
                            break
                    for i in range(len(corner_bounds)):
                        is_connected = False
                        for j in range(len(corner_bounds)):
                            if i==j:
                                self_mask[corner_bounds[i][0]:corner_bounds[i][1],corner_bounds[j][0]:corner_bounds[j][1]] = 0
                            elif any(np.equal([i, 1, j], graph).all(1)) or any(np.equal([j, 1, i], graph).all(1)):
                                door_mask[corner_bounds[i][0]:corner_bounds[i][1],corner_bounds[j][0]:corner_bounds[j][1]] = 0
                                is_connected = True
                        if not is_connected:
                            door_mask[corner_bounds[i][0]:corner_bounds[i][1],corner_bounds[living_room_index][0]:corner_bounds[living_room_index][1]] = 0

                    houses.append(house_layouts)
                    door_masks.append(door_mask)
                    self_masks.append(self_mask)
                    gen_masks.append(gen_mask)
                    graphs.append(graph)
                self.syn_houses = houses
                self.syn_door_masks = door_masks
                self.syn_self_masks = self_masks
                self.syn_gen_masks = gen_masks
                self.syn_graphs = graphs

                # np.savez_compressed(f'processed_rplan/rplan_{set_name}_{target_set}_syn', graphs=self.syn_graphs, houses=self.syn_houses,
                #         door_masks=self.syn_door_masks, self_masks=self.syn_self_masks, gen_masks=self.syn_gen_masks)

                np.savez_compressed(f'processed_rplan/rplan_{set_name}_{target_set}_syn',
                    graphs=np.array(self.syn_graphs, dtype=object),
                    houses=np.array(self.syn_houses, dtype=object),
                    door_masks=np.array(self.syn_door_masks, dtype=object),
                    self_masks=np.array(self.syn_self_masks, dtype=object),
                    gen_masks=np.array(self.syn_gen_masks, dtype=object)
                )

    def __len__(self):
        return len(self.houses)

    def __getitem__(self, idx):
        # idx = int(idx//20)
        arr = self.houses[idx][:, :self.num_coords]
        graph = np.concatenate((self.graphs[idx], np.zeros([200-len(self.graphs[idx]), 3])), 0)

        cond = {
                'door_mask': self.door_masks[idx],
                'self_mask': self.self_masks[idx],
                'gen_mask': self.gen_masks[idx],
                'room_types': self.houses[idx][:, self.num_coords:self.num_coords+25],
                'corner_indices': self.houses[idx][:, self.num_coords+25:self.num_coords+57],
                'room_indices': self.houses[idx][:, self.num_coords+57:self.num_coords+89],
                'src_key_padding_mask': 1-self.houses[idx][:, self.num_coords+89],
                'connections': self.houses[idx][:, self.num_coords+90:self.num_coords+92],
                'graph': graph,
                }
        if self.set_name == 'eval':
            syn_graph = np.concatenate((self.syn_graphs[idx], np.zeros([200-len(self.syn_graphs[idx]), 3])), 0)
            assert (graph == syn_graph).all(), idx
            cond.update({
                'syn_door_mask': self.syn_door_masks[idx],
                'syn_self_mask': self.syn_self_masks[idx],
                'syn_gen_mask': self.syn_gen_masks[idx],
                'syn_room_types': self.syn_houses[idx][:, self.num_coords:self.num_coords+25],
                'syn_corner_indices': self.syn_houses[idx][:, self.num_coords+25:self.num_coords+57],
                'syn_room_indices': self.syn_houses[idx][:, self.num_coords+57:self.num_coords+89],
                'syn_src_key_padding_mask': 1-self.syn_houses[idx][:, self.num_coords+89],
                'syn_connections': self.syn_houses[idx][:, self.num_coords+90:self.num_coords+92],
                'syn_graph': syn_graph,
                })

        if self.set_name == 'train':
            #### Random Rotate
            rotation = random.randint(0,3)
            if rotation == 1:
                arr[:, [0, 1]] = arr[:, [1, 0]]
                arr[:, 0] = -arr[:, 0]
            elif rotation == 2:
                arr[:, [0, 1]] = -arr[:, [1, 0]]
            elif rotation == 3:
                arr[:, [0, 1]] = arr[:, [1, 0]]
                arr[:, 1] = -arr[:, 1]

            ## To generate any rotation uncomment this

            # if self.non_manhattan:
                # theta = random.random()*np.pi/2
                # rot_mat = np.array([[np.cos(theta), -np.sin(theta), 0],
                             # [np.sin(theta), np.cos(theta), 0]])
                # arr = np.matmul(arr,rot_mat)[:,:2]

            # Random Scale
            # arr = arr * np.random.normal(1., .5)

            # Random Shift
            # arr[:, 0] = arr[:, 0] + np.random.normal(0., .1)
            # arr[:, 1] = arr[:, 1] + np.random.normal(0., .1)

        if not self.analog_bit:
            arr = np.transpose(arr, [1, 0])
            return arr.astype(float), cond
        else:
            ONE_HOT_RES = 256
            arr_onehot = np.zeros((ONE_HOT_RES*2, arr.shape[1])) - 1
            xs = ((arr[:, 0]+1)*(ONE_HOT_RES/2)).astype(int)
            ys = ((arr[:, 1]+1)*(ONE_HOT_RES/2)).astype(int)
            xs = np.array([get_bin(x, 8) for x in xs])
            ys = np.array([get_bin(x, 8) for x in ys])
            arr_onehot = np.concatenate([xs, ys], 1)
            arr_onehot = np.transpose(arr_onehot, [1, 0])
            arr_onehot[arr_onehot==0] = -1
            return arr_onehot.astype(float), cond


In [189]:
def make_sequence(self, edges):
        polys = []
        v_curr = tuple(edges[0][:2])
        e_ind_curr = 0
        e_visited = [0]
        seq_tracker = [v_curr]
        find_next = False
        while len(e_visited) < len(edges):
            if find_next == False:
                if v_curr == tuple(edges[e_ind_curr][2:]):
                    v_curr = tuple(edges[e_ind_curr][:2])
                else:
                    v_curr = tuple(edges[e_ind_curr][2:])
                find_next = not find_next
            else:
                # look for next edge
                for k, e in enumerate(edges):
                    if k not in e_visited:
                        if (v_curr == tuple(e[:2])):
                            v_curr = tuple(e[2:])
                            e_ind_curr = k
                            e_visited.append(k)
                            break
                        elif (v_curr == tuple(e[2:])):
                            v_curr = tuple(e[:2])
                            e_ind_curr = k
                            e_visited.append(k)
                            break

            # extract next sequence
            if v_curr == seq_tracker[-1]:
                polys.append(seq_tracker)
                for k, e in enumerate(edges):
                    if k not in e_visited:
                        v_curr = tuple(edges[0][:2])
                        seq_tracker = [v_curr]
                        find_next = False
                        e_ind_curr = k
                        e_visited.append(k)
                        break
            else:
                seq_tracker.append(v_curr)
        polys.append(seq_tracker)

        return polys

RPlanhgDataset.make_sequence = make_sequence

In [190]:
def build_graph(self, rms_type, fp_eds, eds_to_rms, out_size=64):
        # create edges
        triples = []
        nodes = rms_type
        # encode connections
        for k in range(len(nodes)):
            for l in range(len(nodes)):
                if l > k:
                    is_adjacent = any([True for e_map in eds_to_rms if (l in e_map) and (k in e_map)])
                    if is_adjacent:
                        if 'train' in self.set_name:
                            triples.append([k, 1, l])
                        else:
                            triples.append([k, 1, l])
                    else:
                        if 'train' in self.set_name:
                            triples.append([k, -1, l])
                        else:
                            triples.append([k, -1, l])
        # get rooms masks
        eds_to_rms_tmp = []
        for l in range(len(eds_to_rms)):
            eds_to_rms_tmp.append([eds_to_rms[l][0]])
        rms_masks = []
        im_size = 256
        fp_mk = np.zeros((out_size, out_size))
        for k in range(len(nodes)):
            # add rooms and doors
            eds = []
            for l, e_map in enumerate(eds_to_rms_tmp):
                if (k in e_map):
                    eds.append(l)
            # draw rooms
            rm_im = Image.new('L', (im_size, im_size))
            dr = ImageDraw.Draw(rm_im)
            for eds_poly in [eds]:
                poly = self.make_sequence(np.array([fp_eds[l][:4] for l in eds_poly]))[0]
                poly = [(im_size*x, im_size*y) for x, y in poly]
                if len(poly) >= 2:
                    dr.polygon(poly, fill='white')
                else:
                    print("Empty room")
                    exit(0)
            rm_im = rm_im.resize((out_size, out_size))
            rm_arr = np.array(rm_im)
            inds = np.where(rm_arr>0)
            rm_arr[inds] = 1.0
            rms_masks.append(rm_arr)
            if rms_type[k] != 15 and rms_type[k] != 17:
                fp_mk[inds] = k+1

        # trick to remove overlap
        for k in range(len(nodes)):
            if rms_type[k] != 15 and rms_type[k] != 17:
                rm_arr = np.zeros((out_size, out_size))
                inds = np.where(fp_mk==k+1)
                rm_arr[inds] = 1.0
                rms_masks[k] = rm_arr
        # convert to array
        nodes = np.array(nodes)
        triples = np.array(triples)
        rms_masks = np.array(rms_masks)
        return nodes, triples, rms_masks

RPlanhgDataset.build_graph = build_graph

In [191]:
def is_adjacent(box_a, box_b, threshold=0.03):
    x0, y0, x1, y1 = box_a
    x2, y2, x3, y3 = box_b
    h1, h2 = x1-x0, x3-x2
    w1, w2 = y1-y0, y3-y2
    xc1, xc2 = (x0+x1)/2.0, (x2+x3)/2.0
    yc1, yc2 = (y0+y1)/2.0, (y2+y3)/2.0
    delta_x = np.abs(xc2-xc1) - (h1 + h2)/2.0
    delta_y = np.abs(yc2-yc1) - (w1 + w2)/2.0
    delta = max(delta_x, delta_y)
    return delta < threshold

In [192]:
def reader(filename, normalize = True):
    with open(filename) as f:
        info =json.load(f)

        rms_type=info['room_type']
        if normalize == True:
            s_r=0
            for rmk in range(len(rms_type)):
                if(rms_type[rmk]!=17):
                    s_r=s_r+1

        rms_bbs=np.asarray(info['boxes'])
        if normalize == True:
            # from [0,255] to [-1,1]
            rms_bbs = np.array(rms_bbs)/256.0
            tl = np.min(rms_bbs[:, :2], 0)
            br = np.max(rms_bbs[:, 2:], 0)
            shift = (tl+br)/2.0 - 0.5
            rms_bbs[:, :2] -= shift
            rms_bbs[:, 2:] -= shift

        fp_eds=np.asarray(info['edges'])
        if normalize == True:
            # from [0,255] to [-1,1]
            fp_eds = np.array(fp_eds)/256.0
            fp_eds = fp_eds[:, :4]
            fp_eds[:, :2] -= shift
            fp_eds[:, 2:] -= shift

        eds_to_rms=info['ed_rm']
        if normalize == True:
            tl -= shift
            br -= shift

        return rms_type,fp_eds,rms_bbs,eds_to_rms

In [193]:
%pwd

'/content/HOUSE_DIFFUSION_DIFFUSION'

# house_diffusion/image_sampling.py

In [194]:
remove_folder("/content/HOUSE_DIFFUSION_DIFFUSION/outputs")

In [196]:
"""
Generate a large batch of image samples from a model and save them as a large
numpy array. This can be used to produce samples for FID evaluation.
"""

import argparse
import os

import numpy as np
import torch as th

import io
import PIL.Image as Image
import drawsvg
import cairosvg
import imageio
from tqdm import tqdm
import matplotlib.pyplot as plt
from pytorch_fid.fid_score import calculate_fid_given_paths
# from house_diffusion.rplanhg_datasets import load_rplanhg_data
from house_diffusion import dist_util, logger
from house_diffusion.script_util import (
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    add_dict_to_argparser,
    args_to_dict,
    update_arg_parser,
)
import webcolors
import pygraphviz as pgv
import networkx as nx
from collections import defaultdict
from shapely.geometry import Polygon
from shapely.validation import make_valid
from shapely.geometry.base import geom_factory
# from shapely.geos import lgeos

# import random
# th.manual_seed(0)
# random.seed(0)
# np.random.seed(0)

bin_to_int = lambda x: int("".join([str(int(i.cpu().data)) for i in x]), 2)
def bin_to_int_sample(sample, resolution=256):
    sample_new = th.zeros([sample.shape[0], sample.shape[1], sample.shape[2], 2])
    sample[sample<0] = 0
    sample[sample>0] = 1
    for i in range(sample.shape[0]):
        for j in range(sample.shape[1]):
            for k in range(sample.shape[2]):
                sample_new[i, j, k, 0] = bin_to_int(sample[i, j, k, :8])
                sample_new[i, j, k, 1] = bin_to_int(sample[i, j, k, 8:])
    sample = sample_new
    sample = sample/(resolution/2) - 1
    return sample

def get_graph(indx, g_true, ID_COLOR, draw_graph, save_svg):
    # build true graph
    G_true = nx.Graph()
    colors_H = []
    node_size = []
    edge_color = []
    linewidths = []
    edgecolors = []
    # add nodes
    for k, label in enumerate(g_true[0]):
        _type = label
        if _type >= 0 and _type not in [11, 12]:
            G_true.add_nodes_from([(k, {'label':k})])
            colors_H.append(ID_COLOR[_type])
            node_size.append(1000)
            edgecolors.append('blue')
            linewidths.append(0.0)
    # add outside node
    G_true.add_nodes_from([(-1, {'label':-1})])
    colors_H.append("white")
    node_size.append(750)
    edgecolors.append('black')
    linewidths.append(3.0)
    # add edges
    for k, m, l in g_true[1]:
        k = int(k)
        l = int(l)
        _type_k = g_true[0][k]
        _type_l = g_true[0][l]
        if m > 0 and (_type_k not in [11, 12] and _type_l not in [11, 12]):
            G_true.add_edges_from([(k, l)])
            edge_color.append('#D3A2C7')
        elif m > 0 and (_type_k==11 or _type_l==11):
            if _type_k==11:
                G_true.add_edges_from([(l, -1)])
            else:
                G_true.add_edges_from([(k, -1)])
            edge_color.append('#727171')
    if draw_graph:
        plt.figure()
        pos = nx.nx_agraph.graphviz_layout(G_true, prog='neato')
        nx.draw(G_true, pos, node_size=node_size, linewidths=linewidths, node_color=colors_H, font_size=14, font_color='white',\
                font_weight='bold', edgecolors=edgecolors, width=4.0, with_labels=False)
        if save_svg:
            plt.savefig(f'outputs/graphs_gt/{indx}.svg')
        else:
            plt.savefig(f'outputs/graphs_gt/{indx}.jpg')
        plt.close('all')
    return G_true

def estimate_graph(indx, polys, nodes, G_gt, ID_COLOR, draw_graph, save_svg):
    nodes = np.array(nodes)
    G_gt = G_gt[1-th.where((G_gt == th.tensor([0,0,0], device='cuda')).all(dim=1))[0]]
    G_gt = get_graph(indx, [nodes, G_gt], ID_COLOR, draw_graph, save_svg)
    G_estimated = nx.Graph()
    colors_H = []
    node_size = []
    edge_color = []
    linewidths = []
    edgecolors = []
    edge_labels = {}
    # add nodes
    for k, label in enumerate(nodes):
        _type = label
        if _type >= 0 and _type not in [11, 12]:
            G_estimated.add_nodes_from([(k, {'label':k})])
            colors_H.append(ID_COLOR[_type])
            node_size.append(1000)
            linewidths.append(0.0)
    # add outside node
    G_estimated.add_nodes_from([(-1, {'label':-1})])
    colors_H.append("white")
    node_size.append(750)
    edgecolors.append('black')
    linewidths.append(3.0)
    # add node-to-door connections
    doors_inds = np.where((nodes == 11) | (nodes == 12))[0]
    rooms_inds = np.where((nodes != 11) & (nodes != 12))[0]
    doors_rooms_map = defaultdict(list)
    for k in doors_inds:
        for l in rooms_inds:
            if k > l:
                p1, p2 = polys[k], polys[l]
                p1, p2 = Polygon(p1), Polygon(p2)

                # print()
                # print (f"p1: {p1}")
                if not p1.is_valid:
                    # p1 = geom_factory(lgeos.GEOSMakeValid(p1._geom))

                    # print(f"p1._geom: {p1._geom}")
                    p1 = make_valid(p1)

                # print()
                # print (f"p2: {p2}")
                if not p2.is_valid:
                    # p2 = geom_factory(lgeos.GEOSMakeValid(p2._geom))

                    # print(f"p2._geom: {p2._geom}")
                    p2 = make_valid(p2)

                iou = p1.intersection(p2).area/ p1.union(p2).area
                if iou > 0 and iou < 0.2:
                    doors_rooms_map[k].append((l, iou))

    # draw connections
    for k in doors_rooms_map.keys():
        _conn = doors_rooms_map[k]
        _conn = sorted(_conn, key=lambda tup: tup[1], reverse=True)
        _conn_top2 = _conn[:2]
        if nodes[k] != 11:
            if len(_conn_top2) > 1:
                l1, l2 = _conn_top2[0][0], _conn_top2[1][0]
                edge_labels[(l1, l2)] = k
                G_estimated.add_edges_from([(l1, l2)])
        else:
            if len(_conn) > 0:
                l1 = _conn[0][0]
                edge_labels[(-1, l1)] = k
                G_estimated.add_edges_from([(-1, l1)])
    # add missed edges
    G_estimated_complete = G_estimated.copy()
    for k, l in G_gt.edges():
        if not G_estimated.has_edge(k, l):
            G_estimated_complete.add_edges_from([(k, l)])
    # add edges colors
    colors = []
    mistakes = 0
    for k, l in G_estimated_complete.edges():
        if G_gt.has_edge(k, l) and not G_estimated.has_edge(k, l):
            colors.append('yellow')
            mistakes += 1
        elif G_estimated.has_edge(k, l) and not G_gt.has_edge(k, l):
            colors.append('red')
            mistakes += 1
        elif G_estimated.has_edge(k, l) and G_gt.has_edge(k, l):
            colors.append('green')
        else:
            print('ERR')
    if draw_graph:
        plt.figure()
        pos = nx.nx_agraph.graphviz_layout(G_estimated_complete, prog='neato')
        weights = [4 for u, v in G_estimated_complete.edges()]
        nx.draw(G_estimated_complete, pos, edge_color=colors, linewidths=linewidths, edgecolors=edgecolors, node_size=node_size, node_color=colors_H, font_size=14, font_weight='bold', font_color='white', width=weights, with_labels=False)
        if save_svg:
            plt.savefig(f'outputs/graphs_pred/{indx}.svg')
        else:
            plt.savefig(f'outputs/graphs_pred/{indx}.jpg')
        plt.close('all')
    return mistakes

def save_samples(
        sample, ext, model_kwargs,
        tmp_count, num_room_types,
        save_gif=False, save_edges=False,
        door_indices = [11, 12, 13], ID_COLOR=None,
        is_syn=False, draw_graph=False, save_svg=False):

    prefix = 'syn_' if is_syn else ''
    graph_errors = []
    if not save_gif:
        sample = sample[-1:]
    for i in tqdm(range(sample.shape[1])):
        resolution = 256
        images = []
        images2 = []
        images3 = []
        for k in range(sample.shape[0]):
            draw = drawsvg.Drawing(resolution, resolution, displayInline=False)
            draw.append(drawsvg.Rectangle(0,0,resolution,resolution, fill='black'))

            draw2 = drawsvg.Drawing(resolution, resolution, displayInline=False)
            draw2.append(drawsvg.Rectangle(0,0,resolution,resolution, fill='black'))

            draw3 = drawsvg.Drawing(resolution, resolution, displayInline=False)
            draw3.append(drawsvg.Rectangle(0,0,resolution,resolution, fill='black'))

            draw_color = drawsvg.Drawing(resolution, resolution, displayInline=False)
            draw_color.append(drawsvg.Rectangle(0,0,resolution,resolution, fill='white'))

            polys = []
            types = []
            for j, point in (enumerate(sample[k][i])):
                if model_kwargs[f'{prefix}src_key_padding_mask'][i][j]==1:
                    continue
                point = point.cpu().data.numpy()
                if j==0:
                    poly = []
                if j>0 and (model_kwargs[f'{prefix}room_indices'][i, j]!=model_kwargs[f'{prefix}room_indices'][i, j-1]).any():
                    polys.append(poly)
                    types.append(c)
                    poly = []
                pred_center = False
                if pred_center:
                    point = point/2 + 1
                    point = point * resolution//2
                else:
                    point = point/2 + 0.5
                    point = point * resolution
                poly.append((point[0], point[1]))
                c = np.argmax(model_kwargs[f'{prefix}room_types'][i][j-1].cpu().numpy())
            polys.append(poly)
            types.append(c)
            for poly, c in zip(polys, types):
                if c in door_indices or c==0:
                    continue
                room_type = c
                c = webcolors.hex_to_rgb(ID_COLOR[c])
                draw_color.append(drawsvg.Lines(*np.array(poly).flatten().tolist(), close=True, fill=ID_COLOR[room_type], fill_opacity=1.0, stroke='black', stroke_width=1))
                draw.append(drawsvg.Lines(*np.array(poly).flatten().tolist(), close=True, fill='black', fill_opacity=0.0, stroke=webcolors.rgb_to_hex([int(x/2) for x in c]), stroke_width=0.5*(resolution/256)))
                draw2.append(drawsvg.Lines(*np.array(poly).flatten().tolist(), close=True, fill=ID_COLOR[room_type], fill_opacity=1.0, stroke=webcolors.rgb_to_hex([int(x/2) for x in c]), stroke_width=0.5*(resolution/256)))
                for corner in poly:
                    draw.append(drawsvg.Circle(corner[0], corner[1], 2*(resolution/256), fill=ID_COLOR[room_type], fill_opacity=1.0, stroke='gray', stroke_width=0.25))
                    draw3.append(drawsvg.Circle(corner[0], corner[1], 2*(resolution/256), fill=ID_COLOR[room_type], fill_opacity=1.0, stroke='gray', stroke_width=0.25))
            for poly, c in zip(polys, types):
                if c not in door_indices:
                    continue
                room_type = c
                c = webcolors.hex_to_rgb(ID_COLOR[c])
                draw_color.append(drawsvg.Lines(*np.array(poly).flatten().tolist(), close=True, fill=ID_COLOR[room_type], fill_opacity=1.0, stroke='black', stroke_width=1))
                draw.append(drawsvg.Lines(*np.array(poly).flatten().tolist(), close=True, fill='black', fill_opacity=0.0, stroke=webcolors.rgb_to_hex([int(x/2) for x in c]), stroke_width=0.5*(resolution/256)))
                draw2.append(drawsvg.Lines(*np.array(poly).flatten().tolist(), close=True, fill=ID_COLOR[room_type], fill_opacity=1.0, stroke=webcolors.rgb_to_hex([int(x/2) for x in c]), stroke_width=0.5*(resolution/256)))
                for corner in poly:
                    draw.append(drawsvg.Circle(corner[0], corner[1], 2*(resolution/256), fill=ID_COLOR[room_type], fill_opacity=1.0, stroke='gray', stroke_width=0.25))
                    draw3.append(drawsvg.Circle(corner[0], corner[1], 2*(resolution/256), fill=ID_COLOR[room_type], fill_opacity=1.0, stroke='gray', stroke_width=0.25))
            images.append(Image.open(io.BytesIO(cairosvg.svg2png(draw.as_svg()))))
            images2.append(Image.open(io.BytesIO(cairosvg.svg2png(draw2.as_svg()))))
            images3.append(Image.open(io.BytesIO(cairosvg.svg2png(draw3.as_svg()))))
            if k==sample.shape[0]-1 or True:
                if save_edges:
                    draw.save_svg(f'outputs/{ext}/{tmp_count+i}_{k}_{ext}.svg')
                if save_svg:
                    draw_color.save_svg(f'outputs/{ext}/{tmp_count+i}c_{k}_{ext}.svg')
                else:
                    Image.open(io.BytesIO(cairosvg.svg2png(draw_color.as_svg()))).save(f'outputs/{ext}/{tmp_count+i}c_{ext}.png')
            if k==sample.shape[0]-1:
                if 'graph' in model_kwargs:
                    graph_errors.append(estimate_graph(tmp_count+i, polys, types, model_kwargs[f'{prefix}graph'][i], ID_COLOR=ID_COLOR, draw_graph=draw_graph, save_svg=save_svg))

                else:
                    graph_errors.append(0)
        if save_gif:
            imageio.mimwrite(f'outputs/gif/{tmp_count+i}.gif', images, fps=10, loop=1)
            imageio.mimwrite(f'outputs/gif/{tmp_count+i}_v2.gif', images2, fps=10, loop=1)
            imageio.mimwrite(f'outputs/gif/{tmp_count+i}_v3.gif', images3, fps=10, loop=1)
    return graph_errors

def main():
    parser = create_argparser()
    parser.set_defaults(dataset = 'rplan')
    parser.set_defaults(batch_size = 2)
    parser.set_defaults(set_name = 'eval')
    parser.set_defaults(target_set = 8)
    parser.set_defaults(model_path = "/content/HOUSE_DIFFUSION_DIFFUSION/ckpts/exp/model250000.pt")
    parser.set_defaults(num_samples = 5)

    args = parser.parse_args([])
    update_arg_parser(args)

    dist_util.setup_dist()
    logger.configure()

    logger.log("creating model and diffusion...")
    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    model.load_state_dict(
        dist_util.load_state_dict(args.model_path, map_location="cpu")
    )
    model.to(dist_util.dev())
    model.eval()

    errors = []
    for _ in range(1):
        logger.log("sampling...")
        tmp_count = 0
        os.makedirs('outputs/pred', exist_ok=True)
        os.makedirs('outputs/gt', exist_ok=True)
        os.makedirs('outputs/gif', exist_ok=True)
        os.makedirs('outputs/graphs_gt', exist_ok=True)
        os.makedirs('outputs/graphs_pred', exist_ok=True)

        if args.dataset=='rplan':
            ID_COLOR = {1: '#EE4D4D', 2: '#C67C7B', 3: '#FFD274', 4: '#BEBEBE', 5: '#BFE3E8',
                        6: '#7BA779', 7: '#E87A90', 8: '#FF8C69', 10: '#1F849B', 11: '#727171',
                        13: '#785A67', 12: '#D3A2C7'}
            num_room_types = 14
            data = load_rplanhg_data(
                batch_size=args.batch_size,
                analog_bit=args.analog_bit,
                set_name=args.set_name,
                target_set=args.target_set,
                using_memory = False
            )
        else:
            print("dataset does not exist!")
            assert False
        graph_errors = []
        while tmp_count < args.num_samples:
            model_kwargs = {}
            sample_fn = (
                diffusion.p_sample_loop if not args.use_ddim else diffusion.ddim_sample_loop
            )
            data_sample, model_kwargs = next(data)
            for key in model_kwargs:
                model_kwargs[key] = model_kwargs[key].cuda()

            sample = sample_fn(
                model,
                data_sample.shape,
                clip_denoised=args.clip_denoised,
                model_kwargs=model_kwargs,
                analog_bit=args.analog_bit,
            )
            sample_gt = data_sample.cuda().unsqueeze(0)
            sample = sample.permute([0, 1, 3, 2])
            sample_gt = sample_gt.permute([0, 1, 3, 2])
            if args.analog_bit:
                sample_gt = bin_to_int_sample(sample_gt)
                sample = bin_to_int_sample(sample)

            # graph_error = save_samples(sample_gt, 'gt', model_kwargs, tmp_count, num_room_types, ID_COLOR=ID_COLOR, draw_graph=args.draw_graph, save_svg=args.save_svg)
            # graph_error = save_samples(sample, 'pred', model_kwargs, tmp_count, num_room_types, ID_COLOR=ID_COLOR, is_syn=True, draw_graph=args.draw_graph, save_svg=args.save_svg)
            graph_error = save_samples(sample_gt, 'gt', model_kwargs, tmp_count, num_room_types, ID_COLOR=ID_COLOR, draw_graph=args.draw_graph, save_svg=False, save_gif = True)
            graph_error = save_samples(sample, 'pred', model_kwargs, tmp_count, num_room_types, ID_COLOR=ID_COLOR, is_syn=True, draw_graph=args.draw_graph, save_svg=False, save_gif = True)
            graph_errors.extend(graph_error)
            tmp_count+=sample_gt.shape[1]

        logger.log("\n sampling complete")
    #     fid_score = calculate_fid_given_paths(['outputs/gt', 'outputs/pred'], 5, 'cuda', 2048)
    #     print(f'FID: {fid_score}')
    #     print(f'Compatibility: {np.mean(graph_errors)}')
    #     errors.append([fid_score, np.mean(graph_errors)])

    # errors = np.array(errors)
    # print(f'Diversity mean: {errors[:, 0].mean()} \t Diversity std: {errors[:, 0].std()}')
    # print(f'Compatibility mean: {errors[:, 1].mean()} \t Compatibility std: {errors[:, 1].std()}')

def create_argparser():
    defaults = dict(
        dataset='',
        clip_denoised=True,
        num_samples=10000,
        batch_size=16,
        use_ddim=False,
        model_path="",
        draw_graph=True,
        save_svg=True,
    )
    defaults.update(model_and_diffusion_defaults())
    parser = argparse.ArgumentParser()
    add_dict_to_argparser(parser, defaults)
    return parser


if __name__ == "__main__":
    main()


Logging to ckpts/openai_2024_06_11_06_04_19_539236
creating model and diffusion...
Number of model parameters: 26541330
COSINE
sampling...
loading eval of target set 8


100%|██████████| 90/90 [00:01<00:00, 80.99it/s] 
processing dataset: 90it [00:00, 251.32it/s]
processing dataset: 90it [00:00, 146.80it/s]
processing dataset: 90it [00:00, 159.39it/s]


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
100%|██████████| 2/2 [00:07<00:00,  3.85s/it]


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
100%|██████████| 2/2 [00:07<00:00,  3.50s/it]


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
100%|██████████| 2/2 [00:08<00:00,  4.26s/it]


 sampling complete
